In [1]:
import sys
sys.path.append("../../")

import time
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import LSTM, Embedding, RepeatVector, TimeDistributed, Masking, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LambdaCallback

from datasets.unibo_powertools_data import UniboPowertoolsData, CycleCols
from datasets.model_data_handler import ModelDataHandler
from datasets.prepare_rul_data import RulHandler

from utils import metrics
from utils.logger import Logger
import logging

2023-05-27 13:19:18.267920: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-27 13:19:18.306608: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-27 13:19:18.899383: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# 划分训练集和测试集
train_names = [
    '000-DM-3.0-4019-S',#minimum capacity 1.48
    '001-DM-3.0-4019-S',#minimum capacity 1.81
    '002-DM-3.0-4019-S',#minimum capacity 2.06
    '009-DM-3.0-4019-H',#minimum capacity 1.41
    '010-DM-3.0-4019-H',#minimum capacity 1.44
    '014-DM-3.0-4019-P',#minimum capacity 1.7
    '015-DM-3.0-4019-P',#minimum capacity 1.76
    '016-DM-3.0-4019-P',#minimum capacity 1.56
    '017-DM-3.0-4019-P',#minimum capacity 1.29
    #'047-DM-3.0-4019-P',#new 1.98
    #'049-DM-3.0-4019-P',#new 2.19
    '007-EE-2.85-0820-S',#2.5
    '008-EE-2.85-0820-S',#2.49
    '042-EE-2.85-0820-S',#2.51
    '043-EE-2.85-0820-H',#2.31
    '018-DP-2.00-1320-S',#minimum capacity 1.82
    #'019-DP-2.00-1320-S',#minimum capacity 1.61
    '036-DP-2.00-1720-S',#minimum capacity 1.91
    '037-DP-2.00-1720-S',#minimum capacity 1.84
    '038-DP-2.00-2420-S',#minimum capacity 1.854 (to 0)
    '050-DP-2.00-4020-S',#new 1.81
    '051-DP-2.00-4020-S',#new 1.866
    '040-DM-4.00-2320-S',#minimum capacity 3.75, cycles 188
]

test_names = [
    '003-DM-3.0-4019-S',#minimum capacity 1.84
    '011-DM-3.0-4019-H',#minimum capacity 1.36
    '013-DM-3.0-4019-P',#minimum capacity 1.6
    '006-EE-2.85-0820-S',# 2.621
    '044-EE-2.85-0820-H',# 2.43
    '039-DP-2.00-2420-S',#minimum capacity 1.93
    '041-DM-4.00-2320-S',#minimum capacity 3.76, cycles 190
]

CAPACITY_THRESHOLDS = {
  3.0 : 2.7, #th 90% - min 2.1, 70%
  2.85 : 2.7, #th 94.7% - min 2.622, 92%
  2.0 : 1.93, #th 96.5% - min 1.93, 96.5%
  4.0 : 3.77, #th 94.2% - min 3.77 94.2%
  4.9 : 4.7, #th 95.9% - min 4.3, 87.7%
  5.0 : 4.5 #th 90% - min 3.63, 72.6%
}
N_CYCLE = 500
WARMUP_TRAIN = 15
WARMUP_TEST = 30

In [3]:
# 读取数据集
data_root_path = "../../data/raw_data/unibo/"

dataset = UniboPowertoolsData(
    test_types=[],
    chunk_size=1000000,
    lines=[37, 40],
    charge_line=37,
    discharge_line=40,
    base_path=data_root_path
)

dataset.prepare_data(train_names, test_names)
dataset_handler = ModelDataHandler(dataset, [
    CycleCols.VOLTAGE,
    CycleCols.CURRENT,
    CycleCols.TEMPERATURE
])

rul_handler = RulHandler()

(train_x, train_y_soh, test_x, test_y_soh,
  train_battery_range, test_battery_range,
  time_train, time_test, current_train, current_test) = dataset_handler.get_discharge_whole_cycle_future(train_names, test_names)

train_y = rul_handler.prepare_y_future(train_names, train_battery_range, train_y_soh, current_train, time_train, CAPACITY_THRESHOLDS)
del globals()["current_train"]
del globals()["time_train"]
test_y = rul_handler.prepare_y_future(test_names, test_battery_range, test_y_soh, current_test, time_test, CAPACITY_THRESHOLDS)
del globals()["current_test"]
del globals()["time_test"]
train_x, test_x = rul_handler.compress_cycle(train_x, test_x)


x_norm = rul_handler.Normalization()
train_x, test_x = x_norm.fit_and_normalize(train_x, test_x)
train_x = rul_handler.battery_life_to_time_series(train_x, N_CYCLE, train_battery_range)
test_x = rul_handler.battery_life_to_time_series(test_x, N_CYCLE, test_battery_range)

train_x, train_y, train_battery_range, train_y_soh = rul_handler.delete_initial(train_x, train_y, train_battery_range, train_y_soh, WARMUP_TRAIN)
test_x, test_y, test_battery_range, test_y_soh = rul_handler.delete_initial(test_x, test_y, test_battery_range, test_y_soh, WARMUP_TEST)

# first one is SOH, we keep only RUL
train_y = train_y[:,1]
test_y = test_y[:,1]

y_norm = rul_handler.Normalization()
train_y, test_y = y_norm.fit_and_normalize(train_y, test_y)

/home/lxh/battery_phm/notebooks/experiments/../../datasets/model_data_handler.py:91: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x = np.array(


In [4]:
# 模型训练
IS_TRAINING = False
RESULT_NAME = "lstm_rul_unibo_powertools"
model_save_path = "../../checkpoints/unibo_vit_deeplstm/"

if IS_TRAINING:
    EXPERIMENT = "lstm_rul_unibo_powertools"

    experiment_name = time.strftime("%Y-%m-%d-%H-%M-%S") + '_' + EXPERIMENT
    print(experiment_name)

    # Model definition

    opt = tf.keras.optimizers.Adam(lr=0.000003)

    model = Sequential()
    model.add(Masking(input_shape=(train_x.shape[1], train_x.shape[2])))
    model.add(LSTM(128, activation='selu',
                    return_sequences=True,
                    kernel_regularizer=regularizers.l2(0.0002)))
    model.add(LSTM(64, activation='selu', return_sequences=False,
                    kernel_regularizer=regularizers.l2(0.0002)))
    model.add(Dense(64, activation='selu', kernel_regularizer=regularizers.l2(0.0002)))
    model.add(Dense(32, activation='selu', kernel_regularizer=regularizers.l2(0.0002)))
    model.add(Dense(1, activation='linear'))
    model.summary()

    model.compile(optimizer=opt, loss='huber', metrics=['mse', 'mae', 'mape', tf.keras.metrics.RootMeanSquaredError(name='rmse')])

    history = model.fit(train_x, train_y, 
                                epochs=500, 
                                batch_size=32, 
                                verbose=1,
                                validation_split=0
                               )
    
    model.save(model_save_path + '%s.h5' % experiment_name)

    hist_df = pd.DataFrame(history.history)
    hist_csv_file = model_save_path + '%s_history.csv' % experiment_name
    with open(hist_csv_file, mode='w') as f:
        hist_df.to_csv(f)
    history = history.history

if not IS_TRAINING:
    history = pd.read_csv(model_save_path + '%s_history.csv' % RESULT_NAME)
    model = keras.models.load_model(model_save_path + '%s.h5' % RESULT_NAME)
    model.summary()

2023-05-27 13:20:35.303892: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43609 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:65:00.0, compute capability: 8.6


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, 500, 6)            0         
                                                                 
 lstm (LSTM)                 (None, 500, 128)          69120     
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 124,801
Trainable params: 124,801
Non-trai

In [5]:
results = model.evaluate(test_x, test_y, return_dict = True)
mae, rmse = results["mae"], results["rmse"]

test_predictions = model.predict(test_x)

test_y = y_norm.denormalize(test_y)
test_predictions = y_norm.denormalize(test_predictions)

logger = Logger(
        log_root_path="../../logs/",
        log_level=logging.DEBUG,
        logger_name="unibo_deeplstm_cycle"
    ).get_logger()

logger.info(f"average mae: {mae:7.6f}, average rmse: {rmse:7.6f}")

a = 0
for index, b in enumerate(test_battery_range):
    plt.figure(figsize=(8, 6))
    # plt.gca().invert_xaxis()
    plt.plot(test_predictions[a:b, 0])
    plt.plot(test_y[a:b])
    plt.xlabel("循环圈数")
    plt.ylabel("安时剩余寿命（Ah）")
    plt.legend(["预测值", "真值"])
    figure_save_path = f"../../assets/thesis_figures/chapter_5/unibo_lstm_rul_cycle_{index + 1}.jpg"
    plt.savefig(figure_save_path, dpi=1000, bbox_inches="tight")
    plt.clf()

    mse = np.sum((test_y[a:b] - test_predictions[a:b, 0]) ** 2) / len(test_y[a:b])
    rmse = math.sqrt(mse)
    nrmse = rmse / (np.max(test_y[a:b]) - np.min(test_y[a:b]))
    logger.info(f"test dataset: {test_names[index]}, rmse: {rmse:7.6f}, nrmse: {nrmse:7.6f}")
    
    a = b

  1/104 [..............................] - ETA: 1:47 - loss: 0.0095 - mse: 2.4774e-04 - mae: 0.0154 - mape: 8.1123 - rmse: 0.0157

2023-05-27 13:20:37.089041: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


104/104 [==============================] - 13s 118ms/step


2023-05-27 13:21:02,317 - unibo_deeplstm_cycle - INFO - average mae: 0.013523, average rmse: 0.021267


Save log to: ../../logs/2023_05_27_13_21_unibo_deeplstm_cycle.log


...

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>